In [ ]:
import os
import decimal
import yfinance as yf
import time
def save_dict(product_data, crypto=False):
  # Write each dataframe to disk
  for key, df in product_data.items():
    df.to_csv(data_path + "/dqn-preds-" + key + ".csv", index=False)

  # Convert the keys of the dictionary to a dataframe
  keys_df = pd.DataFrame({"keys": list(product_data.keys())})

  # Save the keys dataframe to disk as a CSV file
  if crypto:
    keys_df.to_csv(data_path + "/dqn-preds-keys-crypto.csv", index=False)
  else:
    keys_df.to_csv(data_path + "/dqn-preds-keys.csv", index=False)

def load_dict(crypto=False):
  if crypto:
    keys = pd.read_csv(data_path + "/dqn-preds-keys-crypto.csv")
  else:
    keys = pd.read_csv(data_path + "/dqn-preds-keys.csv")
  product_data = {}
  # Iterate over the keys
  for index, row in keys.iterrows():
    key = row["keys"]
    product_data[key] = pd.read_csv(data_path + "/dqn-preds-" + key + ".csv")
  return product_data

ModuleNotFoundError: ignored

In [ ]:

def get_all_product_timeseries(cutoff=-1, length=320, limit=-1, raise_error=False):
  df_products = get_all_products()
  df_products = df_products[df_products.id.str.endswith('USDT')]
  df_products = df_products[~df_products.id.str.endswith('3L-USDT')]
  df_products = df_products[~df_products.id.str.endswith('3S-USDT')]
  df_products = df_products.sort_values(by="id")
  product_data = {}
  tries = 3
  i = 0
  
  for index, row in df_products.iterrows():
    if cutoff != -1 and i > cutoff:
      return product_data
    else:
      i+=1
    loop = True
    count = 0
    while(loop):
      try:
        time.sleep(0.25)
        print("fetch for: ", row.id)
        df_raw = sort_date(get_coin_data_frames(length, row.id))      
        product_data[row.id] = df_raw
        loop = False
      except Exception as inst:
        if raise_error:
          raise inst
        print("Error: ", inst)
        if "but only got" in str(inst) or "Illegal response" in str(inst):
          loop = False
        else:
          time.sleep(1)
          count = count+1
        if count>tries:
          loop = False
  
  if limit != -1:
    print("Trimming to: ", limit)
    df_products["volValue"] = df_products["volValue"].apply(decimal.Decimal)
    df_products = df_products.sort_values(by = ['volValue'], ascending=False)
    df_products = df_products.head(limit)
    assert(len(df_products) == limit)
    # kill keys not present anymore. this sucks because we spent time downloading
    # these, a better way might be to add symbols when we fail to download one
    # but otherwise we're short symbols and the dqn model wants a specific sized list 
    # (which also sucks and needs to be fixed)
    # loop through the keys of the dictionary
   
    if len(product_data.keys()) > limit:
      print(f"Purging low volume to {limit}")
      for key in list(product_data.keys()):
        # check if the key is in the dataframe
        if key not in df_products['id'].values:
          # if the key is not in the dataframe, remove it from the dictionary
          del product_data[key]
        if len(product_data.keys()) == limit:
          return product_data
  return product_data

In [ ]:

def get_all_stock_timerseries_for_csv(csv_name, bars, cutoff=-1, raise_error=False):
  product_data = {}
  tickers_df = pd.read_csv(data_path + "/" + csv_name)
  total = len(tickers_df)
  i = 0
  for ticker in tickers_df.iloc[:, 0]:
    print("loading ticker: ", ticker, " count:", i, "of:", total)
    if cutoff != -1 and i > cutoff:
      return product_data
    else:
      i+=1 
    try:
      tickerObj = yf.download(tickers = ticker, interval = "1d")
      df = pd.DataFrame(tickerObj)
      if len(df) == 0 or len(df)<bars:
          continue
      df = df.reset_index()
      product_data[ticker] = df.tail(bars)
      time.sleep(0.25)
    except Exception as inst:
      if raise_error:
        raise inst

      print("Error: ", inst)
  return product_data

In [ ]:
logging_level = 2
def error(*args):
    if logging_level >= 0:
        print(*args)

def info(*args):
    if logging_level >=1:
        print(*args)

def verbose(*args):
    if logging_level >=2:
        print(*args)

def debug(*args):
    if logging_level >=3:
        print(*args)

In [ ]:
def append_predictions(product_data, raise_error = False):
  print("making predictions")
  delete_us = []
  for product, df in product_data.items():
    # get the current period bars
    print("predicting:", product)
    if (len(df)<30):
      print("skipping:", product, " not enough bars")
      continue
    # do predictions based on this tick
    try:
      predictions = make_consensus_prediction_raw(df, product)
      product_data[product] = predictions
    except Exception as inst:
      if raise_error:
        raise inst
      print("Error: ", inst)
  return product_data

In [ ]:
me = '/content/drive/My Drive/ml-trde-notebooks/backtest-common.ipynb'
print("Modified")
print(os.stat(me)[-2])
print(os.stat(me).st_mtime)
print(os.path.getmtime(me))